# random notebook to interactively explore issues

In [8]:
# Point Python to the path where we have installed the bad and darc packages
import sys
sys.path.insert(0, '/Users/btvincent/git-local/darc-experiments-python')

In [9]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import seaborn as sns

import darc
from darc.delayed import models
from darc.designs import BayesianAdaptiveDesignGeneratorDARC
from darc.delayed.designs import Griskevicius2011, Frye, Koffarnus_Bickel, Kirby2009

from darc.data_plotting import plot_delay_without_front_end_delays
from darc_parameter_recovery import simulated_experiment_trial_loop

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

## do things...

In [10]:
true_params = pd.DataFrame.from_dict({'logk': [norm(-4.5, 1).rvs()], 'α': [2]})

design_thing = BayesianAdaptiveDesignGeneratorDARC(RA=[10., 20., 30., 40., 50., 60., 70., 80., 90.])
model = models.Hyperbolic(n_particles=5000)
model.θ_true = true_params

In [11]:
fitted_model, summary_stats = simulated_experiment_trial_loop(design_thing, model)

In [5]:
design_thing.all_possible_designs.head()

,RA,DA,PA,RB,DB,PB
0,10.0,0.0,1.0,100.0,0.000694,1.0
1,10.0,0.0,1.0,100.0,0.001389,1.0
2,10.0,0.0,1.0,100.0,0.003472,1.0
3,10.0,0.0,1.0,100.0,0.006944,1.0
4,10.0,0.0,1.0,100.0,0.010417,1.0


In [12]:
design_thing.data.df

,RA,DA,PA,RB,DB,PB,R
0,40.0,0.0,1.0,100.0,120.0,1.0,1
0,50.0,0.0,1.0,100.0,180.0,1.0,1
0,60.0,0.0,1.0,100.0,240.0,1.0,1
0,20.0,0.0,1.0,100.0,2190.0,1.0,1
0,20.0,0.0,1.0,100.0,2920.0,1.0,1
0,30.0,0.0,1.0,100.0,2555.0,1.0,1
0,30.0,0.0,1.0,100.0,3650.0,1.0,1
0,40.0,0.0,1.0,100.0,2920.0,1.0,0
0,40.0,0.0,1.0,100.0,2555.0,1.0,0
0,40.0,0.0,1.0,100.0,2190.0,1.0,1


In [36]:
design_cols = ['RA', 'DA', 'PA', 'RB', 'DB', 'PB']

new = design_thing.data.df.groupby(design_cols).size().reset_index(name='freq')
new['prop'] = design_thing.data.df.groupby(design_cols)['R'].mean().reset_index(name='prop')['prop']

In [37]:
new['prop'] = design_thing.data.df.groupby(design_cols)['R'].mean().reset_index(name='prop')['prop']
new

,RA,DA,PA,RB,DB,PB,freq,prop
0,20.0,0.0,1.0,100.0,2190.0,1.0,1,1.0
1,20.0,0.0,1.0,100.0,2920.0,1.0,1,1.0
2,30.0,0.0,1.0,100.0,2555.0,1.0,1,1.0
3,30.0,0.0,1.0,100.0,3650.0,1.0,1,1.0
4,40.0,0.0,1.0,100.0,120.0,1.0,1,1.0
5,40.0,0.0,1.0,100.0,1825.0,1.0,2,1.0
6,40.0,0.0,1.0,100.0,2190.0,1.0,2,0.5
7,40.0,0.0,1.0,100.0,2555.0,1.0,1,0.0
8,40.0,0.0,1.0,100.0,2920.0,1.0,1,0.0
9,50.0,0.0,1.0,100.0,180.0,1.0,1,1.0


In [39]:
new['x'] = new['DB']
new['y'] = new['RA']/new['RB']
new = new[['x', 'y', 'freq', 'prop']]
new

,x,y,freq,prop
0,2190.0,0.2,1,1.0
1,2920.0,0.2,1,1.0
2,2555.0,0.3,1,1.0
3,3650.0,0.3,1,1.0
4,120.0,0.4,1,1.0
5,1825.0,0.4,2,1.0
6,2190.0,0.4,2,0.5
7,2555.0,0.4,1,0.0
8,2920.0,0.4,1,0.0
9,180.0,0.5,1,1.0


In [49]:
new['prop'].values * [[1], [1], [1]]

array([[1. , 1. , 1. , 1. , 1. , 1. , 0.5, 0. , 0. , 1. , 0.5, 1. , 0. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 0.5, 0. , 0. , 1. , 0.5, 1. , 0. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 0.5, 0. , 0. , 1. , 0.5, 1. , 0. ]])

In [ ]:


# design_thing.data.df['R'] = design_thing.data.df['R'].astype('float64')
# design_thing.data.df['R']

In [ ]:
temp = model.θ['logk']

temp

In [ ]:
temp = model.θ['α']

temp